In [461]:
import pandas as pd
import requests
import os
import shutil
import urllib
from tqdm import tqdm

import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

%matplotlib inline
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
from IPython.display import clear_output, display, HTML

import plotly.express as px


from ipywidgets import interact, interactive, fixed, interact_manual, HBox, VBox, Dropdown
import ipywidgets as widgets





In [302]:
data_path = 'Food Allergies Data'
allergen_paths = {
    'Beef'    : f'{data_path}//beef-and-buffalo-meat-consumption-per-person.csv',
    'Seafood' : f'{data_path}//fish-and-seafood-consumption-per-capita.csv',
    'Cereals' : f'{data_path}//per-capita-consumption-of-cereals-by-commodity-type-daily-kilocalories.csv',
    'Egg'     : f'{data_path}//per-capita-egg-consumption-kilograms-per-year.csv',
    'Milk'    : f'{data_path}//per-capita-milk-consumption.csv',
    'Peanut'  : f'{data_path}//per-capita-peanut-consumption.csv' 
}

In [199]:
allergen_data = {}
for allergen, path in allergen_paths.items():
    df = pd.read_csv(path)
    allergen_data[allergen] = df

In [696]:
df = allergen_data['Egg']
df

,Entity,Code,Year,"Eggs - Food supply quantity (kg/capita/yr) (FAO, 2020)"
0,Afghanistan,AFG,1961,0.94
1,Afghanistan,AFG,1962,1.01
2,Afghanistan,AFG,1963,1.04
3,Afghanistan,AFG,1964,1.06
4,Afghanistan,AFG,1965,1.10
...,...,...,...,...
11023,Zimbabwe,ZWE,2013,1.75
11024,Zimbabwe,ZWE,2014,1.93
11025,Zimbabwe,ZWE,2015,1.84
11026,Zimbabwe,ZWE,2016,1.86


In [697]:
fig = px.choropleth(df, 
    locations= 'Code',
    color= df.columns[-1], 
    hover_name= "Entity", # column to add to hover information
    animation_frame="Year",
    color_continuous_scale=px.colors.sequential.Blues)
# fig.show()

In [698]:
df = df.fillna(0)
fig = px.scatter_geo(df, 
    locations= 'Code',
    color = 'Code', 
    hover_name= "Entity",   # column to add to hover information
    size= df.columns[-1],
    animation_frame= "Year",
    projection= "natural earth")
# fig.show()

In [202]:
fig = px.line(df[df['Code'] == 'BRA'], x='Year', y= df.columns[-1])
fig.show()

In [546]:
@interact(allergen=allergens)

def plot(allergen):

    if allergen != 'None':
        df = allergen_data[allergen]

        fig = px.choropleth(df, 
            locations= 'Code',
            color= df.columns[-1], 
            hover_name= "Entity", # column to add to hover information
            animation_frame="Year",
            color_continuous_scale=px.colors.sequential.Blues,
            labels = {df.columns[-1]: 'consumption'},
            title = df.columns[-1]
                           )
        
        fig.show()

interactive(children=(Dropdown(description='allergen', options=('None', 'Seafood', 'Beef', 'Peanut', 'Milk', '…

# AllerVis

## Use case 1: 
1. Select the allergen 
2. Visualize the level of consumption across all countries 

In [573]:
allergens = ['None', 'Seafood', 'Beef', 'Peanut', 'Milk', 'Egg', 'Cereals']
idioms = ['choropleth', 'scatter_geo']
countries = allergen_data['Beef']['Entity'].unique()

In [668]:
def choropleth(allergen):

    if allergen != 'None':
        df = allergen_data[allergen]

        fig = px.choropleth(df, 
            locations= 'Code',
            color= df.columns[-1], 
            hover_name= "Entity", # column to add to hover information
            animation_frame="Year",
            color_continuous_scale=px.colors.sequential.Blues,
            labels = {df.columns[-1]: 'consumption'},
            title = df.columns[-1]
                           )

        fig.show()
        
def scatter_geo(allergen):
    
    df = allergen_data[allergen].fillna(0)
    fig = px.scatter_geo(df, 
        locations= 'Code',
        # color = 'Code', 
        hover_name= "Entity",   # column to add to hover information
        size= df.columns[-1],
        animation_frame= "Year",
        projection= "natural earth",
        labels = {df.columns[-1]: 'consumption'},
        title = df.columns[-1]
                        )
    fig.show()

In [676]:
allergens_dropdown = Dropdown(
    options=allergens,
    description='Allergen:',
    disabled=False
)

allergens_buttons = widgets.ToggleButtons(
    options=allergens,
    description='Allergen:',
)

idioms_buttons = widgets.ToggleButtons(
    options=idioms,
    description='Idiom:',
)

plot_button_1 = widgets.Button(description='Plot')
reset_button = widgets.Button(description='Reset')


# tab_1 = VBox(children=[allergens_buttons])
# tab_2 = VBox(children=[idioms_buttons])

# tab = widgets.Tab(children=[tab_1, tab_2])
# tab.set_title(0, 'Allergens')
# tab.set_title(1, 'Idioms')

box_1 = VBox(children=[allergens_buttons, idioms_buttons, plot_button_1])

def clear_scene_1():
    clear_output()
    display(box_1)

@plot_button_1.on_click

def plot_on_click_1(b):
    
    clear_scene_1()

    allergen = allergens_dropdown.value
    allergen = allergens_buttons.value
    
    if idioms_buttons.value == 'choropleth':
        choropleth(allergen)
        
    if idioms_buttons.value == 'scatter_geo':
        scatter_geo(allergen)
    
@reset_button.on_click
def reset_on_click(b):
    clear_scene_1()
    

In [677]:
box_1

## Use case 2: 
1. Select the allergen 
2. Select countries
3. Compare temporal trend

In [678]:
def line(allergen, countries):
    
    if allergen != 'None':
        df = allergen_data[allergen]
        
        fig = px.line(df[df['Entity'] == countries[0]], x='Year', y= df.columns[-1])
        fig.show()

In [680]:
countries_dropdown = Dropdown(
    options=countries,
    description='Country:',
    disabled=False
)

add_button = widgets.Button(description='Add')
remove_button = widgets.Button(description='Remove')
plot_button_2 = widgets.Button(description='Plot')

box_2 = VBox(children=[allergens_buttons, countries_dropdown, plot_button_2])

selected_countries = []

def clear_scene_2():
    clear_output()
    display(box_2)
    countries_box = HBox([widgets.Button(description=country) for country in selected_countries])
    display(countries_box)

@add_button.on_click
def add_on_click(b):
    if countries_dropdown.value not in selected_countries:
        selected_countries.append(countries_dropdown.value)
        clear_scene_2()
        # display(countries_box)
    
@remove_button.on_click
def remove_on_click(b):
    if countries_dropdown.value in selected_countries:
        selected_countries.remove(countries_dropdown.value)
        clear_scene_2()
        # display(countries_box)
        

@plot_button_2.on_click        
def plot_on_click_2(b):
    
    clear_scene_2()
    allergen = allergens_buttons.value
    countries = selected_countries
    line(allergen, [countries_dropdown.value])

In [681]:
box_2

HBox()

In [689]:
tab_1 = VBox(children=[box_1])
tab_2 = VBox(children=[box_2])

tab = widgets.Tab(children=[tab_1, tab_2])
tab.set_title(0, 'Maps')
tab.set_title(1, 'Timeseries')

main_box = VBox(children=[tab])

In [692]:
box_1

In [693]:
box_2

HBox()

In [699]:
from Codes.py import get_boxes

ModuleNotFoundError: No module named 'Codes.py'; 'Codes' is not a package